In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import metrics


In [ ]:
round_digits = 4

base_directory = "../results/"
image_results_column = "test_accuracy"
time_results_column = "mae"

In [ ]:
image_classification_results = [
    # Small Image Classification
    [
        [
            "ResNet20-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "ResNet56-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "ResNet110-cifar10-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ResNet20-cifar100-idun-A100-PyTorch-ngc2312.csv",
            "ResNet56-cifar100-idun-A100-PyTorch-ngc2312.csv",
            "ResNet110-cifar100-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ViTS8-cifar10-idun-A100-PyTorch-ngc2312.csv",
            "ViTB8-cifar10-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ViTS8-cifar100-idun-A100-PyTorch-ngc2312.csv",
            "ViTB8-cifar100-idun-A100-PyTorch-ngc2312.csv",
        ],
    ],
    # Large Image Classification
    [
        [
            "ResNet18-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "ResNet50-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "ResNet101-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ResNet18-uc_merced-idun-A100-PyTorch-ngc2312.csv",
            "ResNet50-uc_merced-idun-A100-PyTorch-ngc2312.csv",
            "ResNet101-uc_merced-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ViTTiny16-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "ViTS16-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
            "ViTB16-oxford_flowers102-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ViTTiny16-uc_merced-idun-A100-PyTorch-ngc2312.csv",
            "ViTS16-uc_merced-idun-A100-PyTorch-ngc2312.csv",
            "ViTB16-uc_merced-idun-A100-PyTorch-ngc2312.csv",
        ],
        [
            "ViTTiny16-oxford_flowers102-idun-A100-PyTorch-ngc2312-pretrained.csv",
            "ViTS16-oxford_flowers102-idun-A100-PyTorch-ngc2312-pretrained.csv",
            "ViTB16-oxford_flowers102-idun-A100-PyTorch-ngc2312-pretrained.csv",
        ],
        [
            "ViTTiny16-uc_merced-idun-A100-PyTorch-ngc2312-pretrained.csv",
            "ViTS16-uc_merced-idun-A100-PyTorch-ngc2312-pretrained.csv",
            "ViTB16-uc_merced-idun-A100-PyTorch-ngc2312-pretrained.csv",
        ],
    ],
]

In [ ]:
def save_kde(data, x_label_name, save_path):

    max_y = 0  # Initialize a variable to track the maximum y-axis limit across all plots

    # Set the number of columns and rows for the subplots
    ncols = 2
    nrows = math.ceil(len(data) / ncols)  # Calculate the number of rows needed based on the data size

    # Dynamically adjust figure size based on the number of rows
    figsize = (22, 6 * nrows)

    # Create subplots with the determined number of rows and columns
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    fig.tight_layout(pad=7)  # Add padding between subplots

    # Ensure `ax` is always treated as a 2D array even if there's only one row or column
    ax = np.atleast_2d(ax)

    x_mid = np.zeros((nrows, ncols))  # To store the midpoints of x-axis for each subplot
    x_range = 0  # Initialize variable to track the largest x-axis range

    # Loop through the data and create a KDE plot for each item
    for idx, title in enumerate(data):
        column = idx % ncols  # Determine the column index for the subplot
        row = idx // ncols    # Determine the row index for the subplot

        # Plot KDEs for each model in the inner dictionary of `data[title]`
        for model, values in data[title].items():
            sns.kdeplot(values, ax=ax[row, column], fill=True, label=model)

        ax[row, column].legend(fontsize=25)  # Add a legend with larger font size
        ax[row, column].set_title(title, fontsize=25)  # Set the title for the subplot
        ax[row, column].set_xlabel(x_label_name, fontsize=25)  # Label the x-axis
        ax[row, column].set_ylabel("Probability Density", fontsize=25)  # Label the y-axis
        ax[row, column].tick_params(labelsize=20)  # Adjust tick label sizes

        # Get current x-axis limits and calculate the midpoint
        x_lim = ax[row, column].get_xlim()
        x_mid[row, column] = (x_lim[0] + x_lim[1]) / 2  # Calculate midpoint of the x-axis range

        # Update the largest x-axis range found across all plots
        if (x_lim[1] - x_lim[0]) > x_range:
            x_range = x_lim[1] - x_lim[0]

        # Get current y-axis limits and update the global maximum y-axis limit
        y_lim = ax[row, column].get_ylim()
        if y_lim[1] > max_y:
            max_y = y_lim[1]

    # Set consistent x and y limits for all subplots
    for row_idx in range(nrows):
        for col_idx in range(ncols):
            # Check if the current subplot corresponds to actual data
            if row_idx * ncols + col_idx < len(data):
                # Set x-axis limits for each plot, ensuring the range stays within [0, 100]
                x_min = x_mid[row_idx, col_idx] - x_range / 2
                x_max = x_mid[row_idx, col_idx] + x_range / 2
                if x_min < 0:  # Adjust x_min and x_max if necessary to stay within bounds
                    x_min = 0
                    x_max = x_range
                if x_max > 100:
                    x_min = 100 - x_range
                    x_max = 100

                ax[row_idx, col_idx].set_xlim([x_min, x_max])

                # Set the y-axis limit for each plot to ensure consistency across all plots
                ax[row_idx, col_idx].set_ylim([0, max_y])

    plt.show()  # Display the plots

    # Save the figure to a file, with the file name based on the first three letters of the title
    fig.savefig(save_path + title.replace(" ", "_").replace("(", "").replace(")", "").replace(":", "") + "_kde.pdf", dpi=600, pad_inches=0.1, bbox_inches='tight')


In [ ]:
# Save the summary statistics so they can be saved to a CSV file
summary_statistics = []

for idx, experiment_list in enumerate(image_classification_results):

    experiment_data = {}


    for experiments in experiment_list:
        # Start the title with the experiment number
        title = ""

        # Include model in title
        if "resnet" in experiments[0].lower():
            title += "Model: ResNet"
        elif "vit" in experiments[0].lower():
            title += "Model: ViT"

        # Save the individual results for the plots
        results_values = {}

        # Loop through the individual experiments
        for experiment in experiments:
            df = pd.read_csv(base_directory + "image_classification/" + experiment)

            # Get the 100 results
            results = df[image_results_column].values
            # Calculate the summary statistics
            distribution_mean = metrics.safe_round(metrics.mean(results), round_digits)
            distribution_range = metrics.safe_round(metrics.range(results), round_digits)
            distribution_std = metrics.safe_round(metrics.standard_deviation(results), round_digits)     
            shapiro_p = metrics.safe_round(metrics.shapiro_test(results)[1], round_digits)
            skewness = metrics.safe_round(metrics.skewness(results), round_digits)
            lower_fence_outliers = metrics.lower_fence_outliers(results)
            upper_fence_outliers = metrics.upper_fence_outliers(results)

            # Get the model and dataset from the experiment name
            dataset = experiment.split("-")[1]
            model = experiment.split("-")[0]

            # Get the model parameters
            parameters = metrics.parameters(model)

            # Add the dataset to the title if it is not already there
            if "CIFAR" not in title and "Oxford" not in title and "UC Merced" not in title:
                if dataset == "cifar10":
                    title = title + " Dataset: CIFAR-10"
                elif dataset == "cifar100":
                    title = title + " Dataset: CIFAR-100"
                elif dataset == "oxford_flowers102":
                    title = title + " Dataset: Oxford Flowers"
                elif dataset == "uc_merced":
                    title = title + "Dataset: UC Merced"                

            # Add whether the model is pretrained or not for the ViT 16 models
            if "vit" in model.lower() and "16" in model.lower():
                if "pretrained" in experiment:
                    model += " (Pretrained)"
                    if "pretrained" not in title.lower():
                        title += " (Pretrained)"
                else:
                    model += " (Random)"
                    if "random" not in title.lower():
                        title += " (Random)"

            # Convert the results to percentages
            results = results * 100
            # Save the results for the plot
            results_values[model] = results

            # Save the summary statistics
            summary_statistics.append([dataset, model, parameters, distribution_mean, distribution_range, distribution_std, skewness, shapiro_p, lower_fence_outliers, upper_fence_outliers])

        # Save the results for the experiment
        experiment_data[title] = results_values

    # Plot the results for the experiment as a KDE histogram
    save_kde(experiment_data, "Top-1 Accuracy (%)", "plots/results/ic/")


In [ ]:
summary_statistics_df = pd.DataFrame(summary_statistics, columns=["dataset", "model", "parameters", "mean", "range", "std", "skewness", "shapiro_p", "lower_outliers", "upper_outliers"])
summary_statistics_df.to_csv("image_summary_statistics.csv", index=False)

In [ ]:
time_series_results = [
    [
        ["Autoformer_ETTh1_96_96_0_100.csv", "iTransformer_ETTh1_96_96_0_100.csv", "NLinear_ETTh1_96_96_0_100.csv", "TSMixer_ETTh1_96_96_0_100.csv"],
        ["Autoformer_ETTh1_96_192_0_100.csv", "iTransformer_ETTh1_96_192_0_100.csv", "NLinear_ETTh1_96_192_0_100.csv", "TSMixer_ETTh1_96_192_0_100.csv"],
        ["Autoformer_ETTh1_96_336_0_100.csv", "iTransformer_ETTh1_96_336_0_100.csv", "NLinear_ETTh1_96_336_0_100.csv", "TSMixer_ETTh1_96_336_0_100.csv"],
        ["Autoformer_ETTh1_96_720_0_100.csv", "iTransformer_ETTh1_96_720_0_100.csv", "NLinear_ETTh1_96_720_0_100.csv", "TSMixer_ETTh1_96_720_0_100.csv"],
    ],
    [
        ["Autoformer_ETTh2_96_96_0_100.csv", "iTransformer_ETTh2_96_96_0_100.csv", "NLinear_ETTh2_96_96_0_100.csv", "TSMixer_ETTh2_96_96_0_100.csv"],
        ["Autoformer_ETTh2_96_192_0_100.csv", "iTransformer_ETTh2_96_192_0_100.csv", "NLinear_ETTh2_96_192_0_100.csv", "TSMixer_ETTh2_96_192_0_100.csv"],
        ["Autoformer_ETTh2_96_336_0_100.csv", "iTransformer_ETTh2_96_336_0_100.csv", "NLinear_ETTh2_96_336_0_100.csv", "TSMixer_ETTh2_96_336_0_100.csv"],
        ["Autoformer_ETTh2_96_720_0_100.csv", "iTransformer_ETTh2_96_720_0_100.csv", "NLinear_ETTh2_96_720_0_100.csv", "TSMixer_ETTh2_96_720_0_100.csv"],
    ],
    [
        ["Autoformer_ETTm1_96_96_0_100.csv", "iTransformer_ETTm1_96_96_0_100.csv", "NLinear_ETTm1_96_96_0_100.csv", "TSMixer_ETTm1_96_96_0_100.csv"],
        ["Autoformer_ETTm1_96_192_0_100.csv", "iTransformer_ETTm1_96_192_0_100.csv", "NLinear_ETTm1_96_192_0_100.csv", "TSMixer_ETTm1_96_192_0_100.csv"],
        ["Autoformer_ETTm1_96_336_0_100.csv", "iTransformer_ETTm1_96_336_0_100.csv", "NLinear_ETTm1_96_336_0_100.csv", "TSMixer_ETTm1_96_336_0_100.csv"],
        ["Autoformer_ETTm1_96_720_0_100.csv", "iTransformer_ETTm1_96_720_0_100.csv", "NLinear_ETTm1_96_720_0_100.csv", "TSMixer_ETTm1_96_720_0_100.csv"],
    ],
    [
        ["Autoformer_ETTm2_96_96_0_100.csv", "iTransformer_ETTm2_96_96_0_100.csv", "NLinear_ETTm2_96_96_0_100.csv", "TSMixer_ETTm2_96_96_0_100.csv"],
        ["Autoformer_ETTm2_96_192_0_100.csv", "iTransformer_ETTm2_96_192_0_100.csv", "NLinear_ETTm2_96_192_0_100.csv", "TSMixer_ETTm2_96_192_0_100.csv"],
        ["Autoformer_ETTm2_96_336_0_100.csv", "iTransformer_ETTm2_96_336_0_100.csv", "NLinear_ETTm2_96_336_0_100.csv", "TSMixer_ETTm2_96_336_0_100.csv"],
        ["Autoformer_ETTm2_96_720_0_100.csv", "iTransformer_ETTm2_96_720_0_100.csv", "NLinear_ETTm2_96_720_0_100.csv", "TSMixer_ETTm2_96_720_0_100.csv"],
    ],
    [
        ["Autoformer_traffic_96_96_0_100.csv", "iTransformer_traffic_96_96_0_100.csv", "NLinear_traffic_96_96_0_100.csv", "TSMixer_traffic_96_96_0_100.csv"],
        ["Autoformer_traffic_96_192_0_100.csv", "iTransformer_traffic_96_192_0_100.csv", "NLinear_traffic_96_192_0_100.csv", "TSMixer_traffic_96_192_0_100.csv"],
        ["Autoformer_traffic_96_336_0_100.csv", "NLinear_traffic_96_336_0_100.csv", "TSMixer_traffic_96_336_0_100.csv"],
        ["Autoformer_traffic_96_720_0_100.csv", "NLinear_traffic_96_720_0_100.csv", "TSMixer_traffic_96_720_0_100.csv"],
    ],
    [
        ["Autoformer_weather_96_96_0_100.csv", "iTransformer_weather_96_96_0_100.csv", "NLinear_weather_96_96_0_100.csv", "TSMixer_weather_96_96_0_100.csv"],
        ["Autoformer_weather_96_192_0_100.csv", "iTransformer_weather_96_192_0_100.csv", "NLinear_weather_96_192_0_100.csv", "TSMixer_weather_96_192_0_100.csv"],
        ["Autoformer_weather_96_336_0_100.csv", "iTransformer_weather_96_336_0_100.csv", "NLinear_weather_96_336_0_100.csv", "TSMixer_weather_96_336_0_100.csv"],
        ["Autoformer_weather_96_720_0_100.csv", "iTransformer_weather_96_720_0_100.csv", "NLinear_weather_96_720_0_100.csv", "TSMixer_weather_96_720_0_100.csv"],
    ],
]

In [ ]:
# Save the summary statistics so they can be saved to a CSV file
summary_statistics = []

for idx, experiment_list in enumerate(time_series_results):

    experiment_data = {}

    for experiments in experiment_list:
        # Start the title with the experiment number
        title = ""
        horizon = experiments[1].split("_")[3]

        # Save the individual results for the plots
        results_values = {}

        # Loop through the individual experiments
        for experiment in experiments:
            df = pd.read_csv(base_directory + "time_series/" + experiment)

            # Get the 100 results
            results = df[time_results_column].values
            # Calculate the summary statistics
            distribution_mean = metrics.safe_round(metrics.mean(results), round_digits)
            distribution_range = metrics.safe_round(metrics.range(results), round_digits)
            distribution_std = metrics.safe_round(metrics.standard_deviation(results), round_digits)     
            shapiro_p = metrics.safe_round(metrics.shapiro_test(results)[1], round_digits)
            skewness = metrics.safe_round(metrics.skewness(results), round_digits)
            lower_fence_outliers = metrics.lower_fence_outliers(results)
            upper_fence_outliers = metrics.upper_fence_outliers(results) 

            # Get the model and dataset from the experiment name
            dataset = experiment.split("_")[1]
            model = experiment.split("_")[0]

            # Get the model parameters
            parameters = metrics.parameters(model, dataset, horizon)

            # Add the dataset to the title if it is not already there
            if dataset not in title:
                title = "Dataset: " + dataset + " Horizon: " + horizon

            # Save the results for the plot
            results_values[model] = results

            # Save the summary statistics
            summary_statistics.append([dataset, model, horizon, parameters, distribution_mean, distribution_range, distribution_std, skewness, shapiro_p, lower_fence_outliers, upper_fence_outliers])

        # Save the results for the experiment
        experiment_data[title] = results_values

    # Plot the results for the experiment as a KDE histogram
    save_kde(experiment_data, "Mean Absolute Error", "plots/results/ts/")

In [ ]:
summary_statistics_df = pd.DataFrame(summary_statistics, columns=["dataset", "model", "horizon", "parameters", "mean", "range", "std", "skewness", "shapiro_p", "lower_outliers", "upper_outliers"])
summary_statistics_df.to_csv("time_series_summary_statistics_mae.csv", index=False)